# شبکه‌های عصبی بازگشتی

در ماژول قبلی، ما از نمایش‌های معنایی غنی متن و یک طبقه‌بند خطی ساده بر روی تعبیه‌ها استفاده کردیم. کاری که این معماری انجام می‌دهد این است که معنای کلی کلمات در یک جمله را ثبت می‌کند، اما ترتیب کلمات را در نظر نمی‌گیرد، زیرا عملیات تجمیع روی تعبیه‌ها این اطلاعات را از متن اصلی حذف می‌کند. از آنجا که این مدل‌ها قادر به مدل‌سازی ترتیب کلمات نیستند، نمی‌توانند وظایف پیچیده‌تر یا مبهم‌تری مانند تولید متن یا پاسخ به سوالات را حل کنند.

برای درک معنای دنباله متنی، باید از معماری دیگری از شبکه عصبی استفاده کنیم که به آن **شبکه عصبی بازگشتی** یا RNN گفته می‌شود. در RNN، ما جمله خود را یک نماد در هر زمان از شبکه عبور می‌دهیم و شبکه یک **وضعیت** تولید می‌کند که سپس آن را همراه با نماد بعدی دوباره به شبکه می‌فرستیم.

داده‌های ورودی دنباله‌ای از توکن‌ها $X_0,\dots,X_n$ است. RNN یک دنباله از بلوک‌های شبکه عصبی ایجاد می‌کند و این دنباله را به صورت انتها به انتها با استفاده از پس‌انتشار آموزش می‌دهد. هر بلوک شبکه یک جفت $(X_i,S_i)$ را به عنوان ورودی می‌گیرد و $S_{i+1}$ را به عنوان نتیجه تولید می‌کند. وضعیت نهایی $S_n$ یا خروجی $X_n$ به یک طبقه‌بند خطی ارسال می‌شود تا نتیجه تولید شود. تمام بلوک‌های شبکه وزن‌های یکسانی دارند و با یک پاس پس‌انتشار به صورت انتها به انتها آموزش داده می‌شوند.

از آنجا که بردارهای وضعیت $S_0,\dots,S_n$ از طریق شبکه عبور می‌کنند، شبکه قادر است وابستگی‌های ترتیبی بین کلمات را یاد بگیرد. برای مثال، وقتی کلمه *نه* در جایی از دنباله ظاهر می‌شود، شبکه می‌تواند یاد بگیرد که برخی عناصر درون بردار وضعیت را معکوس کند و در نتیجه نفی ایجاد کند.

> از آنجا که وزن‌های تمام بلوک‌های RNN در تصویر مشترک هستند، می‌توان همان تصویر را به صورت یک بلوک (در سمت راست) با یک حلقه بازگشتی نمایش داد که وضعیت خروجی شبکه را به ورودی بازمی‌گرداند.

بیایید ببینیم چگونه شبکه‌های عصبی بازگشتی می‌توانند به ما در طبقه‌بندی مجموعه داده‌های خبری کمک کنند.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## طبقه‌بند ساده RNN

در مورد RNN ساده، هر واحد بازگشتی یک شبکه خطی ساده است که بردار ورودی و بردار حالت را به صورت ترکیبی دریافت کرده و یک بردار حالت جدید تولید می‌کند. PyTorch این واحد را با کلاس `RNNCell` نمایش می‌دهد و شبکه‌ای از این واحدها را به‌عنوان لایه `RNN` ارائه می‌کند.

برای تعریف یک طبقه‌بند RNN، ابتدا یک لایه تعبیه (embedding) اعمال می‌کنیم تا ابعاد واژگان ورودی کاهش یابد و سپس یک لایه RNN را بر روی آن قرار می‌دهیم:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **توجه:** در اینجا برای سادگی از یک لایه تعبیه‌سازی آموزش‌ندیده استفاده می‌کنیم، اما برای دستیابی به نتایج بهتر می‌توانیم از لایه تعبیه‌سازی پیش‌آموزش‌شده با استفاده از Word2Vec یا GloVe، همان‌طور که در واحد قبلی توضیح داده شد، استفاده کنیم. برای درک بهتر، ممکن است بخواهید این کد را برای کار با تعبیه‌سازی‌های پیش‌آموزش‌شده تطبیق دهید.

در این مثال، ما از یک بارگذار داده با توالی‌های پرشده استفاده خواهیم کرد، بنابراین هر دسته شامل تعدادی توالی پرشده با طول یکسان خواهد بود. لایه RNN توالی‌ای از تنسورهای تعبیه‌سازی را دریافت می‌کند و دو خروجی تولید می‌کند:
* $x$ توالی‌ای از خروجی‌های سلول RNN در هر گام است
* $h$ حالت نهایی پنهان برای آخرین عنصر توالی است

سپس یک طبقه‌بند خطی کاملاً متصل را اعمال می‌کنیم تا تعداد کلاس‌ها را به دست آوریم.

> **توجه:** آموزش RNN‌ها نسبتاً دشوار است، زیرا وقتی سلول‌های RNN در طول توالی باز می‌شوند، تعداد لایه‌های درگیر در پس‌انتشار بسیار زیاد می‌شود. بنابراین باید نرخ یادگیری کوچکی انتخاب کنیم و شبکه را روی مجموعه داده بزرگ‌تری آموزش دهیم تا نتایج خوبی به دست آوریم. این فرآیند ممکن است زمان زیادی ببرد، بنابراین استفاده از GPU ترجیح داده می‌شود.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## حافظه کوتاه‌مدت بلند (LSTM)

یکی از مشکلات اصلی شبکه‌های عصبی بازگشتی کلاسیک، مشکل معروف **گرادیان‌های ناپدیدشونده** است. از آنجا که شبکه‌های بازگشتی در یک مرحله پس‌انتشار به صورت انتها به انتها آموزش داده می‌شوند، انتقال خطا به لایه‌های ابتدایی شبکه دشوار است و در نتیجه شبکه نمی‌تواند روابط بین توکن‌های دور از هم را یاد بگیرد. یکی از روش‌های جلوگیری از این مشکل، معرفی **مدیریت صریح حالت** با استفاده از **دروازه‌ها** است. دو معماری شناخته‌شده در این زمینه عبارتند از: **حافظه کوتاه‌مدت بلند** (LSTM) و **واحد بازپخش دروازه‌دار** (GRU).

![تصویری که یک سلول حافظه کوتاه‌مدت بلند را نشان می‌دهد](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

شبکه LSTM به شکلی مشابه شبکه بازگشتی سازماندهی شده است، اما دو حالت وجود دارد که از لایه‌ای به لایه دیگر منتقل می‌شوند: حالت واقعی $c$ و بردار مخفی $h$. در هر واحد، بردار مخفی $h_i$ با ورودی $x_i$ ترکیب می‌شود و آنها کنترل می‌کنند که چه اتفاقی برای حالت $c$ از طریق **دروازه‌ها** رخ دهد. هر دروازه یک شبکه عصبی با فعال‌سازی سیگموید (خروجی در محدوده $[0,1]$) است که می‌توان آن را به عنوان ماسک بیتی در نظر گرفت وقتی با بردار حالت ضرب می‌شود. دروازه‌های زیر وجود دارند (از چپ به راست در تصویر بالا):
* **دروازه فراموشی** بردار مخفی را می‌گیرد و تعیین می‌کند کدام اجزای بردار $c$ باید فراموش شوند و کدام باید عبور کنند.
* **دروازه ورودی** مقداری اطلاعات از ورودی و بردار مخفی می‌گیرد و آن را وارد حالت می‌کند.
* **دروازه خروجی** حالت را از طریق یک لایه خطی با فعال‌سازی $\tanh$ تبدیل می‌کند، سپس برخی از اجزای آن را با استفاده از بردار مخفی $h_i$ انتخاب می‌کند تا حالت جدید $c_{i+1}$ تولید شود.

اجزای حالت $c$ را می‌توان به عنوان برخی پرچم‌ها در نظر گرفت که می‌توانند روشن و خاموش شوند. برای مثال، وقتی در دنباله‌ای با نام *Alice* مواجه می‌شویم، ممکن است بخواهیم فرض کنیم که به یک شخصیت زن اشاره دارد و پرچمی را در حالت بالا ببریم که نشان دهد یک اسم زنانه در جمله داریم. وقتی در ادامه با عبارت *and Tom* مواجه می‌شویم، پرچمی را بالا می‌بریم که نشان دهد یک اسم جمع داریم. بنابراین با دستکاری حالت، می‌توانیم به طور فرضی ویژگی‌های دستوری بخش‌های جمله را دنبال کنیم.

> **توجه**: یک منبع عالی برای درک ساختار داخلی LSTM این مقاله فوق‌العاده [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) نوشته Christopher Olah است.

در حالی که ساختار داخلی سلول LSTM ممکن است پیچیده به نظر برسد، PyTorch این پیاده‌سازی را در کلاس `LSTMCell` پنهان کرده و شیء `LSTM` را برای نمایش کل لایه LSTM فراهم می‌کند. بنابراین، پیاده‌سازی یک طبقه‌بند LSTM بسیار مشابه شبکه بازگشتی ساده‌ای خواهد بود که در بالا مشاهده کردیم:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## دنباله‌های فشرده‌شده

در مثال ما، مجبور بودیم تمام دنباله‌ها در مینی‌بچ را با بردارهای صفر پر کنیم. این کار باعث اتلاف حافظه می‌شود، اما در RNN‌ها مسئله مهم‌تر این است که سلول‌های اضافی RNN برای آیتم‌های ورودی پرشده ایجاد می‌شوند که در فرآیند آموزش شرکت می‌کنند، اما اطلاعات ورودی مهمی را حمل نمی‌کنند. بهتر است RNN فقط بر اساس اندازه واقعی دنباله آموزش داده شود.

برای این منظور، یک فرمت خاص برای ذخیره دنباله‌های پرشده در PyTorch معرفی شده است. فرض کنید مینی‌بچ ورودی پرشده‌ای داریم که به این شکل است:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
اینجا 0 نشان‌دهنده مقادیر پرشده است، و بردار طول واقعی دنباله‌های ورودی `[5,3,1]` است.

برای اینکه بتوانیم RNN را به‌طور مؤثر با دنباله‌های پرشده آموزش دهیم، می‌خواهیم آموزش گروه اول سلول‌های RNN را با مینی‌بچ بزرگ (`[1,6,9]`) شروع کنیم، اما سپس پردازش دنباله سوم را پایان دهیم و آموزش را با مینی‌بچ‌های کوچک‌تر ادامه دهیم (`[2,7]`, `[3,8]`) و به همین ترتیب. بنابراین، دنباله فشرده‌شده به‌صورت یک بردار نمایش داده می‌شود - در مورد ما `[1,6,9,2,7,3,8,4,5]`، و بردار طول (`[5,3,1]`) که از آن می‌توان به‌راحتی مینی‌بچ پرشده اصلی را بازسازی کرد.

برای تولید دنباله فشرده‌شده، می‌توانیم از تابع `torch.nn.utils.rnn.pack_padded_sequence` استفاده کنیم. تمام لایه‌های بازگشتی، از جمله RNN، LSTM و GRU، از دنباله‌های فشرده‌شده به‌عنوان ورودی پشتیبانی می‌کنند و خروجی فشرده‌شده تولید می‌کنند که می‌توان آن را با استفاده از `torch.nn.utils.rnn.pad_packed_sequence` رمزگشایی کرد.

برای اینکه بتوانیم دنباله فشرده‌شده تولید کنیم، باید بردار طول را به شبکه منتقل کنیم، و بنابراین به تابع متفاوتی برای آماده‌سازی مینی‌بچ‌ها نیاز داریم:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

شبکه واقعی بسیار مشابه `LSTMClassifier` بالا خواهد بود، اما در مرحله `forward` هم دسته‌های کوچک پد شده و هم بردار طول دنباله‌ها دریافت می‌شوند. پس از محاسبه تعبیه، دنباله بسته را محاسبه می‌کنیم، آن را به لایه LSTM ارسال می‌کنیم و سپس نتیجه را دوباره باز می‌کنیم.

> **توجه**: در واقع ما از نتیجه باز شده `x` استفاده نمی‌کنیم، زیرا در محاسبات بعدی از خروجی لایه‌های مخفی استفاده می‌کنیم. بنابراین، می‌توانیم باز کردن را به طور کامل از این کد حذف کنیم. دلیل اینکه آن را اینجا قرار داده‌ایم این است که شما بتوانید این کد را به راحتی تغییر دهید، در صورتی که نیاز داشته باشید از خروجی شبکه در محاسبات بعدی استفاده کنید.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **توجه:** ممکن است متوجه شده باشید که پارامتر `use_pack_sequence` را به تابع آموزش ارسال می‌کنیم. در حال حاضر، تابع `pack_padded_sequence` نیاز دارد که تنسور طول دنباله روی دستگاه CPU باشد، و بنابراین تابع آموزش باید از انتقال داده‌های طول دنباله به GPU هنگام آموزش اجتناب کند. می‌توانید به پیاده‌سازی تابع `train_emb` در فایل [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py) نگاه کنید.


## شبکه‌های بازگشتی دوطرفه و چندلایه

در مثال‌های ما، تمام شبکه‌های بازگشتی تنها در یک جهت عمل می‌کردند، از ابتدای یک دنباله تا انتهای آن. این روش طبیعی به نظر می‌رسد، زیرا شبیه به نحوه خواندن و گوش دادن به گفتار است. با این حال، از آنجا که در بسیاری از موارد عملی دسترسی تصادفی به دنباله ورودی داریم، ممکن است منطقی باشد که محاسبات بازگشتی را در هر دو جهت اجرا کنیم. چنین شبکه‌هایی **شبکه‌های بازگشتی دوطرفه** نامیده می‌شوند و می‌توان آنها را با تنظیم پارامتر `bidirectional=True` در سازنده RNN/LSTM/GRU ایجاد کرد.

هنگام کار با شبکه دوطرفه، به دو بردار حالت مخفی نیاز داریم، یکی برای هر جهت. PyTorch این بردارها را به صورت یک بردار با اندازه دو برابر کدگذاری می‌کند، که بسیار راحت است، زیرا معمولاً حالت مخفی حاصل را به یک لایه خطی کاملاً متصل ارسال می‌کنید و فقط باید این افزایش اندازه را هنگام ایجاد لایه در نظر بگیرید.

شبکه بازگشتی، چه یک‌طرفه و چه دوطرفه، الگوهای خاصی را در یک دنباله ثبت می‌کند و می‌تواند آنها را در بردار حالت ذخیره کند یا به خروجی منتقل کند. همانند شبکه‌های کانولوشنی، می‌توانیم یک لایه بازگشتی دیگر روی لایه اول بسازیم تا الگوهای سطح بالاتری را ثبت کنیم، که از الگوهای سطح پایین استخراج‌شده توسط لایه اول ساخته شده‌اند. این ما را به مفهوم **شبکه بازگشتی چندلایه** می‌رساند، که شامل دو یا چند شبکه بازگشتی است، جایی که خروجی لایه قبلی به عنوان ورودی به لایه بعدی منتقل می‌شود.

![تصویری که یک شبکه بازگشتی چندلایه با حافظه کوتاه‌مدت-بلندمدت را نشان می‌دهد](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*تصویر از [این پست فوق‌العاده](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) نوشته فرناندو لوپز*

PyTorch ساخت چنین شبکه‌هایی را به کاری آسان تبدیل کرده است، زیرا تنها کافی است پارامتر `num_layers` را به سازنده RNN/LSTM/GRU ارسال کنید تا چندین لایه بازگشتی به صورت خودکار ساخته شوند. این همچنین به این معناست که اندازه بردار حالت/مخفی به طور متناسب افزایش می‌یابد و باید این موضوع را هنگام مدیریت خروجی لایه‌های بازگشتی در نظر بگیرید.


## شبکه‌های عصبی بازگشتی (RNN) برای وظایف دیگر

در این بخش، دیدیم که شبکه‌های عصبی بازگشتی می‌توانند برای طبقه‌بندی توالی‌ها استفاده شوند، اما در واقع، آن‌ها می‌توانند وظایف بسیار بیشتری را انجام دهند، مانند تولید متن، ترجمه ماشینی و موارد دیگر. ما این وظایف را در بخش بعدی بررسی خواهیم کرد.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
